# Cotizador
**Puede accederse directamente al cotizador bajo diferentes escenarios en la última celda**
Lo que se desarrollará en las siguientes celdas fue el método  que se empleó para cada función y lograr el cotizador
En general definimos una función por cada elemento o componente que requieren los flujos para funcionar, como lo es el calculo de q_x+t, P_x+t, t_P_x, v^t, Valor Presente Aactuarial, entre otros.

Lo primero que haremos será definir nuestros métodos, tales que usaremos dentro de nuestrra clase:
- Seguro de vida con suma asegurada fija
- Seguro de fallecimiento con suma asegurada fija
- Anualidad anticipada vitalicia 

De los cuales necesitamos datos como:
- tabla
- edad
- temporalidad de cobertura, en este caso será vitalicio 
- plazo de pago de primas
- sumas aseguradas
- diferimiento

# Herramientas
Una de las herramientas principales para nuestros calculos son las herramientas demográficas, por ende, en este caso crearemos **manualmente** los valores y columnas de nuestra TABLA MEDIA (50%), CNSF 2013 M, esto por temas de compatibilidad y cualquiera que quiera usar el código no tenga problema.

Y claro que se puede adaptar a cualquier tabla que se desee utilizar con el siguiente código:
    import pandas as pd

    Ruta al archivo 
    ruta_archivo = "ruta/a/tu/archivo/tasa_mortalidad.xlsx"

    Leer el archivo, en este caso es Excel
    CNSF2013M = pd.read_excel(ruta_archivo)

Observación:
Es importante tener especial cuidado con los nombres de las columnas en las tablas utilizadas para el cálculo de probabilidades, ya que pueden variar dependiendo del origen del archivo o del formato en que se presenten los datos. 

In [4]:
import pandas as pd
import numpy as np

# Crear los datos
edades = list(range(111))
cnsf_m_2013 = [
    0.000433, 0.000433, 0.000434, 0.000434, 0.000435, 0.000436, 0.000438, 0.000440, 0.000443, 0.000446,
    0.000449, 0.000453, 0.000457, 0.000463, 0.000468, 0.000475, 0.000482, 0.000489, 0.000498, 0.000507,
    0.000517, 0.000528, 0.000540, 0.000553, 0.000567, 0.000582, 0.000598, 0.000616, 0.000635, 0.000656,
    0.000678, 0.000703, 0.000729, 0.000757, 0.000788, 0.000821, 0.000857, 0.000896, 0.000938, 0.000983,
    0.001033, 0.001087, 0.001145, 0.001208, 0.001278, 0.001353, 0.001435, 0.001525, 0.001623, 0.001730,
    0.001848, 0.001977, 0.002119, 0.002274, 0.002446, 0.002635, 0.002844, 0.003074, 0.003329, 0.003612,
    0.003926, 0.004275, 0.004664, 0.005096, 0.005579, 0.006119, 0.006723, 0.007400, 0.008160, 0.009015,
    0.009977, 0.011061, 0.012285, 0.013668, 0.015235, 0.017009, 0.019024, 0.021312, 0.023915, 0.026879,
    0.030257, 0.034110, 0.038509, 0.043533, 0.049274, 0.055833, 0.063329, 0.071889, 0.081660, 0.092798,
    0.105476, 0.119875, 0.136184, 0.154594, 0.175291, 0.198441, 0.224184, 0.252613, 0.283760, 0.317576,
    0.353919, 0.392540, 0.433078, 0.475068, 0.517949, 0.561099, 0.603861, 0.645589, 0.685682, 0.723620,
    0.758991
]

# Crear DataFrame
CNSF2013M = pd.DataFrame({
    "Edad": edades,
    "q_x": cnsf_m_2013
})

# PERO FALTARÍA AÑADIR LOS VALORES DE P_x, que por lo general a lo largo del curso nosotros teníamos que calcularlo.
CNSF2013M['p_x'] = 1-CNSF2013M['q_x']

print(CNSF2013M)

     Edad       q_x       p_x
0       0  0.000433  0.999567
1       1  0.000433  0.999567
2       2  0.000434  0.999566
3       3  0.000434  0.999566
4       4  0.000435  0.999565
..    ...       ...       ...
106   106  0.603861  0.396139
107   107  0.645589  0.354411
108   108  0.685682  0.314318
109   109  0.723620  0.276380
110   110  0.758991  0.241009

[111 rows x 3 columns]


Otra de las herramientas necesarias sería definir nuestras funciones para el calculo de los elementos principales de nuestros flujos
- P_x+t
- t_P_x
- q_x
- Valor presente actuarial t_P_x * v^(t+1)

## t_P_x
Los cálculos de t_P_x, que representan probabilidades acumuladas en función del tiempo y de la edad inicial, pueden estructurarse de manera recursiva utilizando una lógica similar a la empleada en la sucesión de Fibonacci. En términos computacionales, la función de Fibonacci ofrece un excelente punto de partida, ya que se basa en resolver un problema a través de la descomposición en subproblemas más simples, los cuales se resuelven utilizando casos base bien definidos. Esta misma estrategia puede aplicarse al cálculo de t_P_X, adaptando la operación central *(que en Fibonacci es una suma)* por una multiplicación acumulativa de probabilidades. 

Así, al definir una función recursiva donde el caso base sea 0_P_x = 1, es posible construir los valores sucesivos como productos del valor anterior y la probabilidad correspondiente en cada periodo. Esta adaptación permite transformar una estructura matemática tradicional en una solución algorítmica clara, reutilizando conceptos bien conocidos y robustos como los que se emplean en la función de Fibonacci.

## q_x
La esencia del cálculo de q_x, entendido como la probabilidad de que una persona de edad x no sobreviva (fallezca) al siguiente periodo o a edad x+1, radica en la obtención precisa de valores específicos dentro de una tabla de datos. Desde el punto de vista computacional, esta búsqueda puede implementarse de forma eficiente utilizando estructuras como los DataFrames de la biblioteca pandas, ampliamente utilizada en análisis de datos en Python. Una de las funcionalidades más potentes de pandas es la capacidad de localizar datos específicos mediante el método .loc[], que permite seleccionar primero la fila *(normalmente asociada a la edad x)* y luego la columna correspondiente al periodo o variable de interés. Esta capacidad de indexación jerárquica facilita la extracción directa y ordenada de los valores de q_x, haciendo posible construir cálculos automatizados y replicables a partir de una tabla base de probabilidades.

# VPA



In [5]:
# Para obtener los valores t_P_x de forma recursiva
# Necesitamos pasarle una lista con los valores de P_x+t para que funcione, en este caso se automatiza dentro de la clase
def CALCULAR_P_xt(edad, t, tabla):
    x_t = np.arange(t+1) + edad
    P_xt = []
    for i in x_t:
        P_xt.append(tabla.loc[tabla['Edad']  == i,'p_x'].item())

    return P_xt

pruebaP_xt = CALCULAR_P_xt(105, 2, CNSF2013M)
print(pruebaP_xt)

# Obtener valores de t_P_x
def CALCULAR_t_P_x(t, P_xt):
    if t == 0:
        return 1
    else:
        return CALCULAR_t_P_x(t - 1, P_xt) * P_xt[t - 1]

# UNA VERSIÓN MEJORADA t_P_x
def CALCULAR_t_P_x_new(t, P_xt):
    if t < 0:
        return 0
    elif t == 0:
        return 1
    elif t > len(P_xt):
        return 0  # fuera del rango de tabla
    else:
        return CALCULAR_t_P_x(t - 1, P_xt) * P_xt[t - 1]

    
# Para obtener los valores Q_x adaptada para servir de forma general
def CALCULAR_Q_x(t, edadContrata, tabla):
    buscar_xt = edadContrata + t
    valor = tabla.loc[tabla['Edad']  == buscar_xt,'q_x'].item()
    return valor

print(CALCULAR_Q_x(1, 1, CNSF2013M))

import numpy as np

def VPA(temporalidad, edad, tabla, tasa):
    valoresT = np.arange(temporalidad+1)
    # Valores que tomará X dentro de los flujos:
    valoresX_edades = edad + valoresT
    # Obtenemos por valores de P_x+t - ESTOS VALORES NOS SIRVEN PARA LA PARTE DE RECURSIVIDAD
    P_xt = []
    for i in valoresX_edades:
        P_xt.append(tabla.loc[tabla['Edad']  == i,'p_x'].item())

    # Función para obtener los valores t_P_x
    list_t_P_x = []
    for i in valoresT:
        list_t_P_x.append(CALCULAR_t_P_x(i, P_xt))
    # Convertimos a un numpy array para facilitar flujos
    t_P_x = np.array(list_t_P_x)
    # print(t_P_x) me daría 6 valores, el 6o valor del array es el que me interesa t_P_x

    # Ahora necesitamos la otra parte de nuestro flujo, los valores presentes
    listaVt = []
    for i in valoresT:
        listaVt.append((1/(1+(tasa/100)))**i)
     #Convertimos esta lista a un numpy array para facilitar los flujos
    Vt = np.array(listaVt)

    # Retornará un flujo => OBTUVIMOS Ax:n^1
    unicoFlujo = t_P_x[-1] * Vt[-1]
    return unicoFlujo
"""
print('PRUEBA DE Ax : n^1 - valor presente actuarial')
prueba = VPA(5, 20, CNSF2013M, 3.5)
print(prueba)
print(type(prueba))
"""


[0.438901, 0.396139, 0.35441100000000003]
0.000434


"\nprint('PRUEBA DE Ax : n^1 - valor presente actuarial')\nprueba = VPA(5, 20, CNSF2013M, 3.5)\nprint(prueba)\nprint(type(prueba))\n"

# Primas Únicas de Riesgo
Para calcular la prima única de riesgo necesitamos calcular las siguientes primas de riesgo:
- Seguro dotal puro (tendrá una suma asegurada)
- Seguro de fallecimiento (tendrá una suma asegurada)
- Anualidad vitalicia anticipada

Para una anualidad anticipada vitalicia, necesitamos conocer la edad máxima definia como omega dentro del curso, esto se puede obtener con la siguiente línea de código:
    omega = CNSF2013M['Edad'].max()

PUR = Ax^1:n+s * SAfallecimiento + 
    Rentas Vitalicias *(Anualidad Vitalicia Anticipada (con edad x+n+s) * SAsupervivencia)* +
    Ax:(x+n)^1 * SAvit + 
    Gastos Leg/Fin

Donde 
n = Periodo de Pago de Primas
s = años de diferimiento (mayor o igual a n, depende del caso claro)

Entonces adaptamos nuestras funciones


In [6]:
def PrimaRiesgo_Fallecimiento(edad, tabla, tasa, diferimiento, SA): # EL DIFERIMIENTO DIRECTAMENTE INCLUYE EL PLAZO DE PRIMAS
    t = diferimiento -1 # desde t = 0 hasta (edad + plazoPrimas + diferimiento) - 1
    lista_Q_x = []
    lista_t_P_x_vt =[]
    for i in np.arange(t): 
        lista_Q_x.append(CALCULAR_Q_x(i, edad, tabla))
        lista_t_P_x_vt.append(VPA(i, edad, tabla, tasa))

    Q_x = np.array(lista_Q_x)
    t_P_x_Vt = np.array(lista_t_P_x_vt)

    flujoContingente = Q_x * t_P_x_Vt
    return flujoContingente.sum() * SA

print('\nPRUEBA DE Ax^1 : n - seguro de fallecimiento')
prueba = PrimaRiesgo_Fallecimiento(19,CNSF2013M, 11, 5, 130000)
print(prueba)
print(type(prueba))
# FUNCIONA



PRUEBA DE Ax^1 : n - seguro de fallecimiento
233.3315951239603
<class 'numpy.float64'>


In [7]:
def rentasVitalicias(SARentas, diferimientoPagoRentas, tabla, edad, tasaAnual): # diferimientoPagoRentas es lo mismo que Diferimiento, resvolví por accidente.
    omega = tabla['Edad'].max()

    tasa_mensual = (1 + tasaAnual / 100) ** (1/12) - 1

    # Supongamos un ejemplo de edad 20, diferimiento de 11

    edadInicioDif = diferimientoPagoRentas + edad # 31

    r = np.arange(12) # [0,1,2,...,11] FIJO
    y = np.arange(edadInicioDif, omega, step=1) # son edades - [31, 32, ..., 109]
    # print('y = ', y)


    paraRec_t = np.arange(edad, omega, step=1) # Para calcular un valor recursivo, necesito P_x+0, P_x+1, P_x+2, etc... esta variable funciona como la parte de x+t y que me ayudará a encontrar las probabilidades que se utilizan para obtener t_P_x.

    # P_xy = P_x+y
    P_xy = [] # No llega hasta omega su prob de morir, pero si es útil para calcular omega_P_x si se requiere
    for i in paraRec_t:
        P_xy.append(tabla.loc[tabla['Edad']  == i,'p_x'].item())

    # Calculamos los elementos de y_P_x, r/12, 1_P_x
    # Esta variable mal escrita "y_new" hace referencia a t en la formula t_P_x, PERO mi función toma a "t" como indice sobre la lista P_x+t y lo calcula, por eso tuve que declarar una nueva "y" el cual toma como índices la sumatoria, entonces este hace alución a la correcta posición que quiero que calcule POR COMO ESTÁ CONSTRUIDA LA FUNCIÓN, en otras palabras accede al t_P_x correcto.

    # Ejemplo, si quería calcular 31_P_20... con la "y" que ya había declarado iba a calcular 51_P_20 mi función, por ende cambie los valores a y_new mal escrito, para que acceda al índice correcto en mi lista P_xi y calcule 31_P_20
    y_new = np.arange(diferimientoPagoRentas, omega - edad+1, step=1)
    valores = []
    for i in y_new:
        # iP_x = y_P_x
        iP_x = CALCULAR_t_P_x_new(i, P_xy)
        # print('y_P_x', iP_x,'con y =', i)
        y_uno_P_x = CALCULAR_t_P_x_new(i+1, P_xy) # y+1_P_x
        for j in r:
            valorPresente = ((1/(1+(tasa_mensual)))**(i*12+r+1))
            valores.append((iP_x - (j/12)*(iP_x - y_uno_P_x )) * valorPresente)


    valores = np.array(valores).sum()

    return SARentas * valores


prueba = rentasVitalicias(30000, 35, CNSF2013M, 20, 11)
print(prueba)
#FUNCIONA

955748.3876033631


In [8]:
def PrimaRiesgo_Sobrevivencia(SA, plazoPrimas, edad, tabla, tasa): # En este caso la temporalidad es el plazo de Primas que tendrá
    VPA_flujo = VPA(plazoPrimas, edad, tabla, tasa)
    return SA * VPA_flujo

print(PrimaRiesgo_Sobrevivencia(50000, 35, 20, CNSF2013M, 3.5))

14438.875487080657


In [9]:
def PrimaRiesgo_Funeraria(SAFun, tabla, edad, tasaAnual): # diferimientoPagoRentas es lo mismo que Diferimiento, resvolví por accidente.
    omega = tabla['Edad'].max()

    tasa_mensual = (1 + tasaAnual / 100) ** (1/12) - 1

    # Supongamos un ejemplo de edad 20, diferimiento de 11

    r = np.arange(12) # [0,1,2,...,11] FIJO
    y = np.arange(0, omega-edad, step=1) # son edades - [0, 1, 2, ..., 110-20]
    # print('y = ', y)


    paraRec_t = np.arange(edad, omega, step=1) # Para calcular un valor recursivo, necesito P_x+0, P_x+1, P_x+2, etc... esta variable funciona como la parte de x+t.

    P_xy = [] # No llega hasta omega su prob de morir, pero si es útil para calcular omega_P_x si se requiere
    for i in paraRec_t:
        P_xy.append(tabla.loc[tabla['Edad']  == i,'p_x'].item())

    # Aqupi no tuve que crear una nueva variable de "y", pues los valores corren desde 0 hasta omega-edad-1 dentro de la sumatoria.
    valores = []
    for i in y:
        iP_x = CALCULAR_t_P_x_new(i, P_xy)
        # print('y_P_x', iP_x,'con y =', i)
        y_uno_P_x = CALCULAR_t_P_x_new(i+1, P_xy)
        for j in r:
            valorPresente = ((1/(1+(tasa_mensual)))**(i*12+r+1))
            valores.append((iP_x - (j/12)*(iP_x - y_uno_P_x )) * ((iP_x - y_uno_P_x)/12) * valorPresente)


    valores = np.array(valores).sum()

    return SAFun * valores

print(PrimaRiesgo_Funeraria(50000, CNSF2013M, 20, 3.5))

46732.23314467555


In [10]:
def PrimaRiesgo_Leg(SALeg, tabla, edad, tasaAnual): # diferimientoPagoRentas es lo mismo que Diferimiento, resvolví por accidente.
    omega = tabla['Edad'].max()

    tasa_mensual = (1 + tasaAnual / 100) ** (1/12) - 1

    # Supongamos un ejemplo de edad 20, diferimiento de 11

    r = np.arange(12) # [0,1,2,...,11] FIJO
    y = np.arange(0, omega-edad, step=1) # son edades - [0, 1, 2, ..., 110-20]
    # print('y = ', y)


    paraRec_t = np.arange(edad, omega, step=1) # Para calcular un valor recursivo, necesito P_x+0, P_x+1, P_x+2, etc... esta variable funciona como la parte de x+t.

    P_xy = [] # No llega hasta omega su prob de morir, pero si es útil para calcular omega_P_x si se requiere
    for i in paraRec_t:
        P_xy.append(tabla.loc[tabla['Edad']  == i,'p_x'].item())

    # Aqupi no tuve que crear una nueva variable de "y", pues los valores corren desde 0 hasta omega-edad-1 dentro de la sumatoria.
    valores = []
    for i in y:
        iP_x = CALCULAR_t_P_x_new(i, P_xy)
        # print('y_P_x', iP_x,'con y =', i)
        y_uno_P_x = CALCULAR_t_P_x_new(i+1, P_xy)
        for j in r:
            valorPresente = ((1/(1+(tasa_mensual)))**(i*12+r+1))
            valores.append((iP_x - (j/12)*(iP_x - y_uno_P_x )) * ((iP_x - y_uno_P_x)/12) * valorPresente)


    valores = np.array(valores).sum()

    return SALeg * valores

print(PrimaRiesgo_Leg(50000, CNSF2013M, 20, 3.5))

46732.23314467555


In [11]:
def rentas_temporales(SARentas, diferimientoPagoRentas, tabla, edad, tasaAnual, temporalidadRentas_Anual): # diferimientoPagoRentas es lo mismo que Diferimiento, resvolví por accidente.
    omega = edad + diferimientoPagoRentas + temporalidadRentas_Anual

    tasa_mensual = (1 + tasaAnual / 100) ** (1/12) - 1

    # Supongamos un ejemplo de edad 20, diferimiento de 11 y pago de rentas por 30 años 

    edadInicioDif = diferimientoPagoRentas + edad # 31

    r = np.arange(12) # [0,1,2,...,11] FIJO
    y = np.arange(edadInicioDif, omega, step=1) # son edades - [31, 32, ..., 60]
    # print('y = ', y)


    paraRec_t = np.arange(edad, omega, step=1) # Para calcular un valor recursivo, necesito P_x+0, P_x+1, P_x+2, etc... esta variable funciona como la parte de x+t y que me ayudará a encontrar las probabilidades que se utilizan para obtener t_P_x.

    # P_xy = P_x+y
    P_xy = [] # No llega hasta omega su prob de morir, pero si es útil para calcular omega_P_x si se requiere
    for i in paraRec_t:
        P_xy.append(tabla.loc[tabla['Edad']  == i,'p_x'].item())

    # Calculamos los elementos de y_P_x, r/12, 1_P_x
    # Esta variable mal escrita "y_new" hace referencia a t en la formula t_P_x, PERO mi función toma a "t" como indice sobre la lista P_x+t y lo calcula, por eso tuve que declarar una nueva "y" el cual toma como índices la sumatoria, entonces este hace alución a la correcta posición que quiero que calcule POR COMO ESTÁ CONSTRUIDA LA FUNCIÓN, en otras palabras accede al t_P_x correcto.

    # Ejemplo, si quería calcular 31_P_20... con la "y" que ya había declarado iba a calcular 51_P_20 mi función, por ende cambie los valores a y_new mal escrito, para que acceda al índice correcto en mi lista P_xi y calcule 31_P_20
    y_new = np.arange(diferimientoPagoRentas, omega - edad+1, step=1)
    valores = []
    for i in y_new:
        # iP_x = y_P_x
        iP_x = CALCULAR_t_P_x_new(i, P_xy)
        # print('y_P_x', iP_x,'con y =', i)
        y_uno_P_x = CALCULAR_t_P_x_new(i+1, P_xy) # y+1_P_x
        for j in r:
            valorPresente = ((1/(1+(tasa_mensual)))**(i*12+r+1))
            valores.append((iP_x - (j/12)*(iP_x - y_uno_P_x )) * valorPresente)


    valores = np.array(valores).sum()

    return SARentas * valores


prueba = rentas_temporales(20000, 45, CNSF2013M, 20, 11, 30)
print(prueba)

198415.40665546557


In [12]:
def CALCULO_rentas(SARentas, diferimientoPagoRentas, tabla, edad, tasaAnual, temporalidadRentas_Anual): # 0 = Vitalicio, 1 u otro valor rentas temporales, limitadas por el usuario
    if temporalidadRentas_Anual == 0:
        omega = tabla['Edad'].max()
        tasa_mensual = (1 + tasaAnual / 100) ** (1/12) - 1


        edadInicioDif = diferimientoPagoRentas + edad 

        r = np.arange(12) 
        y = np.arange(edadInicioDif, omega, step=1) 

        paraRec_t = np.arange(edad, omega, step=1) 

        P_xy = [] 
        for i in paraRec_t:
            P_xy.append(tabla.loc[tabla['Edad']  == i,'p_x'].item())

    
        y_new = np.arange(diferimientoPagoRentas, omega - edad+1, step=1)
        valores = []
        for i in y_new:
            iP_x = CALCULAR_t_P_x_new(i, P_xy)
            y_uno_P_x = CALCULAR_t_P_x_new(i+1, P_xy) 
            for j in r:
                valorPresente = ((1/(1+(tasa_mensual)))**(i*12+r+1))
                valores.append((iP_x - (j/12)*(iP_x - y_uno_P_x )) * valorPresente)


        valores = np.array(valores).sum()
        return SARentas * valores
    elif temporalidadRentas_Anual > 0:
        omega = edad + diferimientoPagoRentas + temporalidadRentas_Anual

        tasa_mensual = (1 + tasaAnual / 100) ** (1/12) - 1

        edadInicioDif = diferimientoPagoRentas + edad 

        r = np.arange(12) 
        y = np.arange(edadInicioDif, omega, step=1)


        paraRec_t = np.arange(edad, omega, step=1)

        P_xy = [] 
        for i in paraRec_t:
            P_xy.append(tabla.loc[tabla['Edad']  == i,'p_x'].item())

        y_new = np.arange(diferimientoPagoRentas, omega - edad+1, step=1)
        valores = []
        for i in y_new:
            iP_x = CALCULAR_t_P_x_new(i, P_xy)
            y_uno_P_x = CALCULAR_t_P_x_new(i+1, P_xy)
            for j in r:
                valorPresente = ((1/(1+(tasa_mensual)))**(i*12+r+1))
                valores.append((iP_x - (j/12)*(iP_x - y_uno_P_x )) * valorPresente)


        valores = np.array(valores).sum()

        return SARentas * valores
    
prueba = CALCULO_rentas(50000, 45, CNSF2013M, 20, 3.5, 30)
print(prueba)

21249079.462887153


In [13]:
def CALCULADOR_PUR(edad, tabla, tasaAnual, diferimiento, plazoPrimas, SAF, SAS, SARentas):
    PUR_F = PrimaRiesgo_Fallecimiento(edad, tabla, tasaAnual, diferimiento, SAF)
    rentas = rentasVitalicias(SARentas, diferimiento, tabla, edad, tasaAnual) # La conversión de la tasa a mensual ya se realiza dentro de la función
    PUR_S = PrimaRiesgo_Sobrevivencia(SAS, plazoPrimas, edad, tabla, tasaAnual) 
    PUR = PUR_F + rentas + PUR_S
    return PUR

PUR_PRUEBA = CALCULADOR_PUR(20, CNSF2013M, 3.5, 45, 10, 100000, 50000, 10000)
print(PUR_PRUEBA)

4342375.090086106


# Calculo de Prima Neta Nivelada
Para el calculo de esta prima y para este seguro, necesitamos construir lo siguiente:
- Función que calcule la anualidad temporal anticipada para nivelar nuestra PUR a la temporalidad deseada.
- Función que calcule la PUR, que en este caso es la suma de todas nuestras primas de riesgo, el cual podemos construirla de dos formas, directamente como una variable igual a la suma de los resultados o crear una sola función que calcule todas las primas de riesgo...

In [14]:
def anualidad_Temp_Anticipada(tabla, tasa = float, anualidadTemporalidad = int, edad = int):
    #RECUERDA QUE LOS FLUJOS serán obtenidos por LA FUNCIÓN "VPA"
    valoresTAnualidad = np.arange(anualidadTemporalidad) #+1
    tasa = tasa/100

    #Construimos la sumatoria, recuerda que la función VPA solo retorna un flujo y una anualdiad de este tipo (TA) esta construida con base a una suma de flujos...
    #Esos flujos, esos valores que me retornará la función "VPA" serán guardados en un numpy array para despues simplemente sumarlos (obtenemos flujos y aplicamos sumatoria)
    valores = []
    for i in valoresTAnualidad:
        valores .append(VPA(i, edad, tabla, tasa))

    valoresFlujo = np.array(valores)
    valorAnualidad = float(valoresFlujo.sum())

    return valorAnualidad

In [15]:
def PNN_directa(edad, tabla, tasaAnual, diferimiento, plazoPrimas, SAF, SAS, SARentas):
    PUR = CALCULADOR_PUR(edad, tabla, tasaAnual, diferimiento, plazoPrimas, SAF, SAS, SARentas)
    flujoAnualidad = anualidad_Temp_Anticipada(tabla, tasaAnual, plazoPrimas, edad)

    return (PUR/flujoAnualidad)

print(PNN_directa(20, CNSF2013M, 3.5, 45, 44, 100000, 50000, 10000))

100938.6143531937


# CALCULO DE PRIMA DE TARIFA
Para ello necesitamos los siguientes elementos:
- Función que calcule el valor de alpha%, beta% y gamma% que coorresponden a las comisiones en porcentaje de los gastos de adquisición, operación y margen de ganancia... 
- Función directa que calcula la PNN

In [16]:
def CALCULADOR_per_gastos(edad, tabla, tasaPoliza, nivelacion, intereses = list):
    # Trabaremos de la misma forma sobre vectores/numpy arrays
    # La temporalidad se basará en que se pagan 
    plazo = len(intereses)
    t = np.arange(plazo)

    valorest_P_x_vt =[]
    # Al tabajar con la variable "t" dentro del siguiente ciclo nos aseguramos de trabajar bajo un mismo/igual tamaño de listas, que representan nuestros flujos
    for i in t:
        valorest_P_x_vt.append(VPA(i, edad, tabla, tasaPoliza))
    
    t_P_x_vt = np.array(valorest_P_x_vt)

    # O BIEN PUDIMOS HABER USADO LA FUNCIÓN DE ANUALIDAD Y AHORAR MAS CODIGO PERO REDUCIR CODIGO NO IMPLICA UNA MEJOR FUNCIONALIDAD EN ESTE CASO PUES IMAGINA, QUE PASA SI AHORA ES BAJO UN SEGURO DE VIDA, SI ES BAJO UNA PUR, ya no necsitamos en ese caso del todo una sumatoria con la cual se basa una anualidad, por ende dejamos así el código.


    # Si la tasa que se usaba en v^t correspondía al interes del gasto establecida durante t, hubieramos usado el siguiente código:
    # listaVt = [(1 / (1 + (intereses[i] / 100)))**i for i in t]
    # Vt = np.array(listaVt)

    #Necesito ahora transformar la lista de intereses en un numpy array debido a que se utilizan dentro del flujo.
    tasas = np.array(intereses)/100
    #print(tasas)

    #Ahora creamos nuestros flujos
    # FLUJO DEL NUMERADOR
    flujoNumereador = tasas * t_P_x_vt
    numerador = float(flujoNumereador.sum())

    # FLUJO DENOMINADOR
    # El flujo denominador por construcción de nuestra formula de prima de tarifa, trabaja sobre la temporalidad = plazo de primas, no sobre la temporalidad de las tasas de gastos!
    # ESTO NO ES MÁS QUE UNA ANUALIDAD TEMPORAL ANTICIPADA
    anualidad = anualidad_Temp_Anticipada(tabla, tasaPoliza, nivelacion, edad)

    tasaFinal = numerador/anualidad
    # print(numerador,'/', anualidad)

    return tasaFinal

In [17]:
gastosADQ = [8,2,1]
gastosADM = [5,4,3,2,1]
gastosMU = [3,3,2,2,2]

def PT(PR, edad, tabla, tasaPoliza, temporalidadADQ, temporalidadADM, temporalidadMG,  gastosA = list, gastosO= list, gastosMU= list):
    porcentajeA = (CALCULADOR_per_gastos(edad, tabla, tasaPoliza, temporalidadADQ, gastosA,)) # RETORNA EL PORCENTAJE EN DECIMAL
    porcentajeO = (CALCULADOR_per_gastos(edad, tabla, tasaPoliza, temporalidadADM, gastosO))
    porcentajeMU = (CALCULADOR_per_gastos(edad, tabla, tasaPoliza, temporalidadMG, gastosMU))
    return (PR/(1-porcentajeA-porcentajeO-porcentajeMU))

print(PT(PUR_PRUEBA, 20, CNSF2013M, 3.5, 10, 50, 50, gastosADQ, gastosADM, gastosMU))
print(PUR_PRUEBA)

4414006.473598632
4342375.090086106


# DEFINIMOS NUESTRA CLASE



In [24]:
class Cotizador:
	# Creamos el constructor
	def __init__(self, tabla = CNSF2013M, edad=30, tasaAnual=5.15, diferimiento=45, plazoPrimas=15, SAF=500000, SAS=4800, SARentas=8500, SAFun = 22000, SALeg = 7500, temporalidadADQ = 1, temporalidadADM = 1, temporalidadMG = 1,  gastosA = [16], gastosO= [5], gastosMU= [5], plazoPagoRentas = 0):
		# Plazo primas se usa para el calculo de primas de riesgo, si se desea calcular PRIMA UNICA, el plazo de primas es 1 
		self.__tabla = tabla
		self.__edad = edad
		self.__tasaAnual = tasaAnual
		self.__diferimiento = diferimiento
		self.__plazoPrimas = plazoPrimas
		self.__SAF = SAF
		self.__SAS = SAS
		self.__SARentas = SARentas
		self.__SAFun = SAFun
		self.__SALeg = SALeg
		self.__temporalidadADQ = temporalidadADQ
		self.__temporalidadADM = temporalidadADM
		self.__temporalidadMG = temporalidadMG
		self.__gastosA = gastosA
		self.__gastosO = gastosO
		self.__gastosMU = gastosMU
		self.__plazoPagoRentas = plazoPagoRentas
		

	# Getters

	@property
	def tabla(self):
		return self.__tabla

	@property
	def edad(self):
		return self.__edad

	@property
	def tasaAnual(self):
		return self.__tasaAnual

	@property
	def diferimiento(self):
		return self.__diferimiento

	@property
	def plazoPrimas(self):
		return self.__plazoPrimas

	@property
	def SAF(self):
		return self.__SAF

	@property
	def SAS(self):
		return self.__SAS

	@property
	def SARentas(self):
		return self.__SARentas
	
	@property
	def SAFun(self):
		return self.__SAFun
	
	@property
	def SALeg(self):
		return self.__SALeg
	
	@property
	def temporalidadADQ(self):
		return self.__temporalidadADQ	

	@property
	def temporalidadADM(self):
		return self.__temporalidadADM

	@property
	def temporalidadMG(self):
		return self.__temporalidadMG

	@property
	def gastosA(self):
		return self.__gastosA
	
	@property
	def gastosO(self):
		return self.__gastosO
	
	@property
	def gastosMU(self):
		return self.__gastosMU
	
	@property
	def plazoPagoRentas(self):
		return self.__plazoPagoRentas
	

	# Setters

	@tabla.setter
	def tabla(self, tabla):
		self.__tabla = tabla

	@edad.setter
	def edad(self, edad):
		self.__edad = edad

	@tasaAnual.setter
	def tasaAnual(self, tasaAnual):
		self.__tasaAnual = tasaAnual

	@diferimiento.setter
	def diferimiento(self, diferimiento):
		self.__diferimiento = diferimiento

	@plazoPrimas.setter
	def plazoPrimas(self, plazoPrimas):
		self.__plazoPrimas = plazoPrimas

	@SAF.setter
	def SAF(self, SAF):
		self.__SAF = SAF

	@SAS.setter
	def SAS(self, SAS):
		self.__SAS = SAS

	@SARentas.setter
	def SARentas(self, SARentas):
		self.__SARentas = SARentas

	@SAFun.setter
	def SAFun(self, SAFun):
		self.__SAFun = SAFun

	@SALeg.setter
	def SALeg(self, SALeg):
		self.__SALeg = SALeg

	@temporalidadADQ.setter
	def temporalidadADQ(self, temporalidadADQ):
		self.__temporalidadADQ = temporalidadADQ

	@temporalidadADM.setter
	def temporalidadADM(self, temporalidadADM):
		self.__temporalidadADM = temporalidadADM

	@temporalidadMG.setter
	def temporalidadMG(self, temporalidadMG):
		self.__temporalidadMG = temporalidadMG

	@gastosA.setter
	def gastosA(self, gastosA):
		self.__gastosA = gastosA

	@gastosO.setter
	def gastosO(self, gastosO):
		self.__gastosO = gastosO

	@gastosMU.setter
	def gastosMU(self, gastosMU):
		self.__gastosMU = gastosMU

	@plazoPagoRentas.setter
	def plazoPagoRentas(self, plazoPagoRentas):
		self.__plazoPagoRentas = plazoPagoRentas

	

	# MÉTODOS CALCULADORES

	def PrimaRiesgo_Fallecimiento(self):
		t = self.__edad + self.__diferimiento
		lista_Q_x = []
		lista_t_P_x_vt = []
		for i in np.arange(t): 
			lista_Q_x.append(CALCULAR_Q_x(i, self.__edad, self.__tabla))
			lista_t_P_x_vt.append(VPA(i, self.__edad, self.__tabla, self.__tasaAnual))

		Q_x = np.array(lista_Q_x)
		t_P_x_Vt = np.array(lista_t_P_x_vt)

		flujoContingente = Q_x * t_P_x_Vt
		
		return flujoContingente.sum() * self.__SAS
	
	def rentasVitalicias(self):
		omega = self.__tabla['Edad'].max()

		tasa_mensual = (1 + self.__tasaAnual / 100) ** (1/12) - 1

		edadInicioDif = self.__diferimiento + self.__edad

		r = np.arange(12)
		y = np.arange(edadInicioDif, omega, step=1)

		paraRec_t = np.arange(self.__edad, omega, step=1)

		P_xy = []
		for i in paraRec_t:
			P_xy.append(self.__tabla.loc[self.__tabla['Edad'] == i, 'p_x'].item())

		y_new = np.arange(self.__diferimiento, omega - self.__edad + 1, step=1)
		valores = []
		for i in y_new:
			iP_x = CALCULAR_t_P_x_new(i, P_xy)
			y_uno_P_x = CALCULAR_t_P_x_new(i+1, P_xy)
			for j in r:
				valorPresente = ((1/(1+(tasa_mensual)))**(i*12 + j + 1))
				valores.append((iP_x - (j/12)*(iP_x - y_uno_P_x)) * valorPresente)

		valores = np.array(valores).sum()

		return self.__SARentas * valores
	
	def CALCULO_rentas(self): # El problema con este código es su complejidad algorítmica, no es eficiente,
		if self.__plazoPagoRentas == 0:  # Renta vitalicia
			omega = self.__tabla['Edad'].max()

			tasa_mensual = (1 + self.__tasaAnual / 100) ** (1/12) - 1

			edadInicioDif = self.__diferimiento + self.__edad

			r = np.arange(12)
			y = np.arange(edadInicioDif, omega, step=1)

			paraRec_t = np.arange(self.__edad, omega, step=1)

			P_xy = []
			for i in paraRec_t:
				P_xy.append(self.__tabla.loc[self.__tabla['Edad'] == i, 'p_x'].item())

			y_new = np.arange(self.__diferimiento, omega - self.__edad + 1, step=1)
			valores = []
			for i in y_new:
				iP_x = CALCULAR_t_P_x_new(i, P_xy)
				y_uno_P_x = CALCULAR_t_P_x_new(i+1, P_xy)
				for j in r:
					valorPresente = ((1/(1+(tasa_mensual)))**(i*12 + j + 1))
					valores.append((iP_x - (j/12)*(iP_x - y_uno_P_x)) * valorPresente)

			valores = np.array(valores).sum()

			return self.__SARentas * valores

		else:  # Renta temporal
			omega = self.__edad + self.__diferimiento + self.__plazoPagoRentas
			tasa_mensual = (1 + self.__tasaAnual / 100) ** (1 / 12) - 1

			edadInicioDif = self.__diferimiento + self.__edad
			r = np.arange(12)
			y = np.arange(edadInicioDif, omega, step=1)
			paraRec_t = np.arange(self.__edad, omega, step=1)

			P_xy = []
			for i in paraRec_t:
				P_xy.append(self.__tabla.loc[self.__tabla['Edad'] == i, 'p_x'].item())

			y_new = np.arange(self.__diferimiento, omega - self.__edad + 1, step=1)
			valores = []
			for i in y_new:
				iP_x = CALCULAR_t_P_x_new(i, P_xy)
				y_uno_P_x = CALCULAR_t_P_x_new(i + 1, P_xy)
				for j in r:
					valorPresente = ((1 / (1 + tasa_mensual)) ** (i * 12 + j + 1))
					valores.append((iP_x - (j / 12) * (iP_x - y_uno_P_x)) * valorPresente)

			valores = np.array(valores).sum()

			return self.__SARentas * valores

	
	def PrimaRiesgo_Sobrevivencia(self):
		VPA_flujo = VPA(self.__plazoPrimas, self.__edad, self.__tabla, self.__tasaAnual)
		return self.__SAS * VPA_flujo
	
	def PrimaRiesgo_Funeraria(self):
		omega = self.__tabla['Edad'].max()
		tasa_mensual = (1 + self.__tasaAnual / 100) ** (1 / 12) - 1

		r = np.arange(12)
		y = np.arange(0, omega - self.__edad, step=1)
		paraRec_t = np.arange(self.__edad, omega, step=1)

		P_xy = []
		for i in paraRec_t:
			P_xy.append(self.__tabla.loc[self.__tabla['Edad'] == i, 'p_x'].item())

		valores = []
		for i in y:
			iP_x = CALCULAR_t_P_x_new(i, P_xy) # Estos métodos no son de la clase, por ende se llaman sin el self.
			y_uno_P_x = CALCULAR_t_P_x_new(i + 1, P_xy)
			for j in r:
				valorPresente = (1 / (1 + tasa_mensual)) ** (i * 12 + j + 1)
				valores.append(
					(iP_x - (j / 12) * (iP_x - y_uno_P_x)) *
					((iP_x - y_uno_P_x) / 12) *
					valorPresente
				)

		valores = np.sum(valores)
		return self.__SAFun * valores


	
	def CALCULADOR_PUR(self):
		return (
			PrimaRiesgo_Fallecimiento(
				self.__edad,
				self.__tabla,
				self.__tasaAnual,
				self.__diferimiento,
				self.__SAF
			)
			+ CALCULO_rentas(
				self.__SARentas,
				self.__diferimiento,
				self.__tabla,
				self.__edad,
				self.__tasaAnual,
				self.__plazoPagoRentas
			)
			+ PrimaRiesgo_Sobrevivencia(
				self.__SAS,
				self.__plazoPrimas,
				self.__edad,
				self.__tabla,
				self.__tasaAnual
			)
			+ PrimaRiesgo_Funeraria(
				self.__SAFun,
				self.__tabla,
				self.__edad,
				self.__tasaAnual,
			)
			+ PrimaRiesgo_Leg(
				self.__SALeg,
				self.__tabla,
				self.__edad,
				self.__tasaAnual,
			)
		)
	
	'''def CALCULADOR_PUR(self):
		return (
			self.PrimaRiesgo_Fallecimiento()
			+ self.CALCULO_rentas()
			+ self.PrimaRiesgo_Sobrevivencia()
			+ self.PrimaRiesgo_Funeraria())'''
		
	
	def PNN_directa(self):
		PUR = self.CALCULADOR_PUR()
		flujoAnualidad = anualidad_Temp_Anticipada(
			self.__tabla,
			self.__tasaAnual,
			self.__plazoPrimas,
			self.__edad
		)
		PNN = PUR / flujoAnualidad
		return PNN
	
	def CALCULADOR_PT(self):
		PR = self.PNN_directa()  # Usamos el método de la clase

		porcentajeA = CALCULADOR_per_gastos(
			self.__edad,
			self.__tabla,
			self.__tasaAnual,
			self.__temporalidadADQ,
			self.__gastosA
		)

		porcentajeO = CALCULADOR_per_gastos(
			self.__edad,
			self.__tabla,
			self.__tasaAnual,
			self.__temporalidadADM,
			self.__gastosO
		)

		porcentajeMU = CALCULADOR_per_gastos(
			self.__edad,
			self.__tabla,
			self.__tasaAnual,
			self.__temporalidadMG,
			self.__gastosMU
		)

		return PR / (1 - porcentajeA - porcentajeO - porcentajeMU)

		"""
		def __str__(self):
		Método que permite imprimir un Cotizador en formato cadena con resultados clave.
		:return: Resultados de PUR, PNN y PT.
		:rtype: str
		
		try:
			pur = self.CALCULADOR_PUR()
			pnn = self.PNN_directa()
			pt = self.CALCULADOR_PT()
			
			resultado = (
            "Resultados del Cotizador:\n"
            "PUR (Prima Única de Riesgo): ${:,.2f}\n"
            "PNN (Prima Neta Nivelada):   ${:,.2f} anuales\n"
            "PT  (Prima Tarifa):          ${:,.2f} anuales\n".format(pur, pnn, pt)
			)
		except Exception as e:
			resultado = f"Error al calcular valores: {e}"

		return resultado
		"""

	def __str__(self):
		"""
		Método que permite imprimir un Cotizador en formato cadena con resultados clave,
		incluyendo desglose de la Prima Única de Riesgo (PUR).
		:return: Resultados del seguro.
		:rtype: str
		"""
		try:
			nombre_seguro = self.__nombreSeguro if hasattr(self, "_Cotizador__nombreSeguro") else "Nombre no definido"
			
			# Cálculos desglosados de PUR
			riesgo_fallecer = PrimaRiesgo_Fallecimiento(
				self.__edad,
				self.__tabla,
				self.__tasaAnual,
				self.__diferimiento,
				self.__SAF
			)
			riesgo_sobrevivir = PrimaRiesgo_Sobrevivencia(
				self.__SAS,
				self.__plazoPrimas,
				self.__edad,
				self.__tabla,
				self.__tasaAnual)
			rentas = CALCULO_rentas(
				self.__SARentas,
				self.__diferimiento,
				self.__tabla,
				self.__edad,
				self.__tasaAnual,
				self.__plazoPagoRentas)
			funeraria = PrimaRiesgo_Funeraria(
				self.__SAFun,
				self.__tabla,
				self.__edad,
				self.__tasaAnual,
			)
			legales = PrimaRiesgo_Leg(
				self.__SALeg,
				self.__tabla,
				self.__edad,
				self.__tasaAnual,
			)

			pur_total = self.CALCULADOR_PUR()
			# riesgo_fallecer + riesgo_sobrevivir + rentas + funeraria + legales

			# Otros cálculos
			pnn = self.PNN_directa()
			pt = self.CALCULADOR_PT()

			resultado = (
				f"Nombre del Seguro: {'Vitanova'}\n"
				f"Resultados del Cotizador:\n"
				f"PR (Prima de Riesgo): 	     ${pur_total:,.2f}\n"
				f"  ├── Riesgo de Fallecer:        ${riesgo_fallecer:,.2f}\n"
				f"  ├── Gasto para Estudios:       ${riesgo_sobrevivir:,.2f}\n"
				f"  ├── Rentas:                    ${rentas:,.2f}\n"
				f"  |── Gastos Funerarios:         ${funeraria:,.2f}\n"
				f"  └── Gastos Legales:            ${legales:,.2f}\n"
				f"Número de pago de primas (anuales): {self.__plazoPrimas}\n"
				f"PNN (Prima Neta Nivelada):   ${pnn:,.2f} anuales\n"
				f"PT  (Prima Tarifa):          ${pt:,.2f} anuales\n"
			)
		except Exception as e:
			resultado = f"Error al calcular valores: {e}"

		return resultado






# Resultados - 
Recordemos que 
- En el parámetro de plazoPrimas - 1 si se desea calcular Prima Única de Riesgo
- En el parámetro de plazoPagoRentas - 0 si es vitalicio

En este caso obtendremos las primas con los valores definidos dentro de la clase como predeterminados.

In [25]:
obj = Cotizador()
print(obj)


Nombre del Seguro: Vitanova
Resultados del Cotizador:
PR (Prima de Riesgo): 	     $1,090,440.37
  ├── Riesgo de Fallecer:        $17,436.72
  ├── Gasto para Estudios:       $2,228.75
  ├── Rentas:                    $1,050,934.50
  |── Gastos Funerarios:         $14,796.22
  └── Gastos Legales:            $5,044.17
Número de pago de primas (anuales): 15
PNN (Prima Neta Nivelada):   $73,374.54 anuales
PT  (Prima Tarifa):          $99,154.79 anuales



# Cotizador final y observaciones.
Aquí definiremos nuestro propio ejemplo, se pueden cambiar parámetros.

Se puede observar dentro que las primas de riesgo de superviviencia, fallecimiento y funeraria quizá son un poco bajas (se cobra poco), pero es debido a que usamos la misma tasa definida en todos los calculos.
Para dicha tasa, se usa una tasa libre de riesgo, en general me basé en la tasa establecida en CETES, el cual varía entre el 8 y 11%, al usar esta tasa nuestras primas de riesgo bajan **PERO si queremos cobrar más**, entonces **bajamos el porcentaje de la tasa** como 3.5%, aquí obtendrémos una prima de riesgo más alta y tiene sentido.
Esto se logra observar con el cotizador.

Para los parámetros:
- En el parámetro de plazoPrimas - 1 si se desea calcular Prima Única de Riesgo
- En el parámetro de plazoPagoRentas - 0 si es vitalicio

In [29]:
cot = Cotizador(
    tabla=CNSF2013M,
    edad=40,
    tasaAnual=5.15, 
    diferimiento=25, # Incluye plazo de pago de primas
    plazoPrimas=1, # 1 ES PARA PRIMA ÚNICA
    SAF=500000,
    SAS=4800,
    SARentas=8500,
    SAFun=30000,
    SALeg=30000,
    temporalidadADQ=1,
    temporalidadADM=1,
    temporalidadMG=1,
    gastosA= [5], # Lo ideal  es que se anoten todos los porcentajes, si la temporalidad es de 5 entonces que haya 5 valores dentro de esta lista
    gastosO=[5],
    gastosMU=[5], # Si hay más o menos valores de porcentajes dentro de la lista que la temporalidad del gasto indicada, no falla el código.
    plazoPagoRentas=0 # 0 ES PARA VITALICIO
)

print(cot)

Nombre del Seguro: Vitanova
Resultados del Cotizador:
PR (Prima de Riesgo): 	     $4,305,063.27
  ├── Riesgo de Fallecer:        $14,193.21
  ├── Gasto para Estudios:       $4,560.19
  ├── Rentas:                    $4,226,084.53
  |── Gastos Funerarios:         $30,112.67
  └── Gastos Legales:            $30,112.67
Número de pago de primas (anuales): 1
PNN (Prima Neta Nivelada):   $4,305,063.27 anuales
PT  (Prima Tarifa):          $5,064,780.32 anuales

